# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [34]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math
from scipy import stats
from scipy.stats import percentileofscore as score
from secrets import IEX_CLOUD_API_TOKEN

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data.status_code
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'GL(AORAANMKGETLTS)DBN EASSQEC L/ ',
 'calculationPrice': 'close',
 'open': 0,
 'openTime': None,
 'openSource': 'facliiof',
 'close': 0,
 'closeTime': None,
 'closeSource': 'oiciffal',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 126.79,
 'latestSource': 'Close',
 'latestTime': 'June 4, 2021',
 'latestUpdate': 1692361836640,
 'latestVolume': None,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 127.78,
 'previousVolume': 79545523,
 'change': 2.37,
 'changePercent': 0.01954,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': None,
 'avgTotalVolume': 84590798,
 'iexBidPrice': No

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
#     print(data.status_code)
#     print(data)
    for symbol in symbol_string.split(','):
#         print(symbol)
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'], 
                    'N/A'
                ], 
                index=my_columns), 
            ignore_index=True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,138.10,47.63,N/A
1,AAL,24.30,-1.69,N/A
2,AAP,194.35,20.81,N/A
3,AAPL,131.19,28.74,N/A
4,ABBV,115.47,39.62,N/A
...,...,...,...,...
95,CINF,125.96,6.82,N/A
96,CL,84.04,27.77,N/A
97,CLX,180.07,25.92,N/A
98,CMA,81.62,13.1,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0].copy()
final_dataframe = final_dataframe[:50].copy()
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,604.81,4.44,N/A
1,CINF,125.96,6.82,N/A
2,AFL,58.25,7.61,N/A
3,CE,174.49,9.36,N/A
4,C,82.68,10.96,N/A
5,CI,258.32,11.07,N/A
6,ALL,141.03,11.99,N/A
7,CMA,81.62,13.1,N/A
8,BK,54.52,14.01,N/A
9,CFG,50.10,14.06,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
position_size = float(portfolio_size)/len(final_dataframe.index)
# position_size

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

# final_dataframe['Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,604.81,4.44,33
1,CINF,125.96,6.82,158
2,AFL,58.25,7.61,343
3,CE,174.49,9.36,114
4,C,82.68,10.96,241
5,CI,258.32,11.07,77
6,ALL,141.03,11.99,141
7,CMA,81.62,13.1,245
8,BK,54.52,14.01,366
9,CFG,50.10,14.06,399


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [48]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data.status_code)
# print(data[symbol]['advanced-stats'])


# Price-to-earnings ratio - quote
pe_ratio = data[symbol]['quote']['peRatio']
                      
# Price-to-book ratio - advanced-stats
pb_ratio = data[symbol]['advanced-stats']['priceToBook'] #np.NaN

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales'] #np.NaN

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] #np.NaN
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
# print(ev_to_ebitda)

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [56]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-To-earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price_to_Sales Ratio',
    'PS Percentile',    
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [57]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        # Price-to-earnings ratio - quote
        pe_ratio = data[symbol]['quote']['peRatio']
                      
        # Price-to-book ratio - advanced-stats
        pb_ratio = data[symbol]['advanced-stats']['priceToBook'] 

        # Price-to-sales ratio
        ps_ratio = data[symbol]['advanced-stats']['priceToSales'] 

        # Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] 
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        # Enterprise Value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    pe_ratio,
                    'N/A',
                    pb_ratio,
                    'N/A',
                    ps_ratio,
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ], 
                index=rv_columns), 
            ignore_index=True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.40,N/A,46.67,N/A,8.8,N/A,7.47,N/A,29.644086,N/A,13.707742,N/A,N/A
1,AAL,25.00,N/A,-1.71,N/A,-1.97,N/A,1.21,N/A,-3.856566,N/A,3.236710,N/A,N/A
2,AAP,202.03,N/A,21.23,N/A,3.6,N/A,1.21,N/A,10.987279,N/A,2.742588,N/A,N/A
3,AAPL,129.48,N/A,29.5,N/A,30.57,N/A,6.47,N/A,21.400506,N/A,16.302565,N/A,N/A
4,ABBV,115.54,N/A,40.12,N/A,15.1,N/A,4.01,N/A,12.492506,N/A,8.442384,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.48,N/A,31.92,N/A,-4.47,N/A,6.06,N/A,23.875630,N/A,10.652915,N/A,N/A
501,ZBH,161.98,N/A,59.21,N/A,2.74,N/A,4.75,N/A,27.858643,N/A,8.110482,N/A,N/A
502,ZBRA,538.81,N/A,45.07,N/A,11.93,N/A,6.11,N/A,32.958619,N/A,13.233625,N/A,N/A
503,ZION,60.82,N/A,7.21,N/A,1.29,N/A,2.75,N/A,5.873169,N/A,2.752418,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,251.13,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,293.23,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,30.11,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.34,N/A,14.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,65.78,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.72,N/A,32.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.75,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,197.59,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2533.71,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [59]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-To-earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price_to_Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

In [60]:
for column in ['Price', 'Price-To-earnings Ratio', 'Price-to-Book Ratio', 'Price_to_Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [61]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [62]:
metrics = {
    'Price-To-earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price_to_Sales Ratio': 'PS Percentile',    
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
         rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.40,N/A,46.67,0.805941,8.80,0.780198,7.47,0.79505,29.644086,0.809901,13.707742,0.778218,N/A
1,AAL,25.00,N/A,-1.71,0.122772,-1.97,0.0851485,1.21,0.115842,-3.856566,0.0376238,3.236710,0.130693,N/A
2,AAP,202.03,N/A,21.23,0.376238,3.60,0.487129,1.21,0.115842,10.987279,0.233663,2.742588,0.10495,N/A
3,AAPL,129.48,N/A,29.50,0.528713,30.57,0.954455,6.47,0.748515,21.400506,0.639604,16.302565,0.855446,N/A
4,ABBV,115.54,N/A,40.12,0.742574,15.10,0.875248,4.01,0.538614,12.492506,0.324752,8.442384,0.487129,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.48,N/A,31.92,0.580198,-4.47,0.0831683,6.06,0.723762,23.875630,0.69901,10.652915,0.661386,N/A
501,ZBH,161.98,N/A,59.21,0.861386,2.74,0.377228,4.75,0.590099,27.858643,0.794059,8.110482,0.471287,N/A
502,ZBRA,538.81,N/A,45.07,0.79604,11.93,0.835644,6.11,0.731683,32.958619,0.873267,13.233625,0.764356,N/A
503,ZION,60.82,N/A,7.21,0.132673,1.29,0.130693,2.75,0.376238,5.873169,0.0594059,2.752418,0.106931,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [63]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics:
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.40,N/A,46.67,0.805941,8.80,0.780198,7.47,0.79505,29.644086,0.809901,13.707742,0.778218,0.793861
1,AAL,25.00,N/A,-1.71,0.122772,-1.97,0.0851485,1.21,0.115842,-3.856566,0.0376238,3.236710,0.130693,0.0984158
2,AAP,202.03,N/A,21.23,0.376238,3.60,0.487129,1.21,0.115842,10.987279,0.233663,2.742588,0.10495,0.263564
3,AAPL,129.48,N/A,29.50,0.528713,30.57,0.954455,6.47,0.748515,21.400506,0.639604,16.302565,0.855446,0.745347
4,ABBV,115.54,N/A,40.12,0.742574,15.10,0.875248,4.01,0.538614,12.492506,0.324752,8.442384,0.487129,0.593663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.48,N/A,31.92,0.580198,-4.47,0.0831683,6.06,0.723762,23.875630,0.69901,10.652915,0.661386,0.549505
501,ZBH,161.98,N/A,59.21,0.861386,2.74,0.377228,4.75,0.590099,27.858643,0.794059,8.110482,0.471287,0.618812
502,ZBRA,538.81,N/A,45.07,0.79604,11.93,0.835644,6.11,0.731683,32.958619,0.873267,13.233625,0.764356,0.800198
503,ZION,60.82,N/A,7.21,0.132673,1.29,0.130693,2.75,0.376238,5.873169,0.0594059,2.752418,0.106931,0.161188


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [64]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe = rv_dataframe[:50].copy()
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.20,N/A,8.21,0.136634,0.6342,0.0891089,0.4886,0.0217822,4.168845,0.0475248,0.472548,0.00990099,0.0609901
1,MCK,195.60,N/A,-7.00,0.10297,-1485.5900,0.0039604,0.1316,0.00990099,7.474712,0.0950495,2.639293,0.10297,0.0629703
2,AIV,7.28,N/A,-82.20,0.0217822,2.0700,0.287129,-33.2900,0.0019802,-44.444078,0.00792079,-53.424957,0.00594059,0.0649505
3,L,60.30,N/A,-426.02,0.0039604,0.9176,0.0990099,1.1000,0.0990099,8.593508,0.126733,1.079804,0.029703,0.0716832
4,AIG,54.84,N/A,-12.12,0.0891089,0.7414,0.0930693,1.0700,0.0940594,6.782031,0.0673267,1.043910,0.0277228,0.0742574
5,HFC,35.96,N/A,-38.00,0.049505,1.1000,0.110891,0.5266,0.0277228,8.799768,0.138614,2.981440,0.118812,0.0891089
6,ABC,120.19,N/A,-6.41,0.108911,-87.6900,0.00792079,0.1244,0.00792079,9.098008,0.148515,4.293150,0.209901,0.0966337
7,AAL,25.00,N/A,-1.71,0.122772,-1.9700,0.0851485,1.2100,0.115842,-3.856566,0.0376238,3.236710,0.130693,0.0984158
8,BA,253.69,N/A,-12.18,0.0871287,-8.3800,0.039604,2.7100,0.372277,-19.982874,0.019802,-43.003022,0.00792079,0.105347
9,VIAC,42.10,N/A,9.27,0.142574,1.4200,0.160396,1.0400,0.0910891,7.272751,0.0851485,1.491793,0.0514851,0.106139


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [65]:
portfolio_input()

Enter the value of your portfolio:2500000


In [66]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
#print(position_size) 

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-To-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price_to_Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,32.20,1552,8.21,0.136634,0.6342,0.0891089,0.4886,0.0217822,4.168845,0.0475248,0.472548,0.00990099,0.0609901
1,MCK,195.60,255,-7.00,0.10297,-1485.5900,0.0039604,0.1316,0.00990099,7.474712,0.0950495,2.639293,0.10297,0.0629703
2,AIV,7.28,6868,-82.20,0.0217822,2.0700,0.287129,-33.2900,0.0019802,-44.444078,0.00792079,-53.424957,0.00594059,0.0649505
3,L,60.30,829,-426.02,0.0039604,0.9176,0.0990099,1.1000,0.0990099,8.593508,0.126733,1.079804,0.029703,0.0716832
4,AIG,54.84,911,-12.12,0.0891089,0.7414,0.0930693,1.0700,0.0940594,6.782031,0.0673267,1.043910,0.0277228,0.0742574
5,HFC,35.96,1390,-38.00,0.049505,1.1000,0.110891,0.5266,0.0277228,8.799768,0.138614,2.981440,0.118812,0.0891089
6,ABC,120.19,416,-6.41,0.108911,-87.6900,0.00792079,0.1244,0.00792079,9.098008,0.148515,4.293150,0.209901,0.0966337
7,AAL,25.00,2000,-1.71,0.122772,-1.9700,0.0851485,1.2100,0.115842,-3.856566,0.0376238,3.236710,0.130693,0.0984158
8,BA,253.69,197,-12.18,0.0871287,-8.3800,0.039604,2.7100,0.372277,-19.982874,0.019802,-43.003022,0.00792079,0.105347
9,VIAC,42.10,1187,9.27,0.142574,1.4200,0.160396,1.0400,0.0910891,7.272751,0.0851485,1.491793,0.0514851,0.106139


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [79]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [80]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [81]:
columns_format = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-To-earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price_to_Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template]
}
    

for column in columns_format.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, columns_format[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', columns_format[column][0], columns_format[column][1])
    

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [82]:
writer.save()